In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import sys
import re
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored

sys.path.append("../")
import dataset_handler as dh
import helper_func as hf

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [10]:
name = "FA_gsm8k_fewer_alt_codegen_func"

file = "data/priming_texts/gsm8k/"+ name +".txt"


with open(file, "r") as f:
    prompt = f.read()
#gptj_model = "EleutherAI/gpt-j-6B"

In [3]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
#model_args.model = "codegen-350M-mono"
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 499.37s
loading tokenizer
loading tokenizer took 4.08s


In [ ]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [28]:
def preproc_gen_toks(gen_toks, input_len, tokenizer):
    """Process generated tokens from model keeping only up to \n\n

    :param list of list gen_toks: the output form the model
    :param int input_len: input lenght used to ignore the prompt
    :param HF_tokenizer tokenizer: tokenizer used for decoding
    :return list of str : list of generated outputs
    """
    list_out = []
    for gen_tok in gen_toks:
        last_tokens = gen_tok[input_len:]
        generated_text = tokenizer.decode(last_tokens)
        #print(generated_text)
        print_pattern = re.compile(r"float\(([^)]+)\)")
        split_list = re.split(print_pattern, generated_text)
        if len(split_list) > 1:
            output = f"{split_list[0]}float({split_list[1]})\n"
        else:
            output = "INVALID OUTPUT"
        # output = generated_text.split("\n\n")[0]
        list_out.append(output)
    return list_out

In [71]:
def verify_pred_from_output(output, sample_a, print_output=False):
        """Verify the the output generates the solution

        :param str output: output generated by the language model
        :param str sample_a: str of solution, should be castable to float otherwise it will be changed to default wrong value
        :return bool: True if the output generates the solution, False otherwise
        """
        if print_output:
            print(colored(f"Return Sequence:", "yellow"))

        avoid_input = re.compile(r"input\(([^)]+)\)")
        if avoid_input.search(output):
            pass
        else:
            try:
                exec(f"def exercise8():{output}\n", globals())
                s = exercise8()
                if print_output:
                    print(colored(f"{s}", "yellow"))
            except Exception as e:
                s = 1111111111.0

        is_correct = s == sample_a
        if print_output:
            print(colored(f"{output}", "green" if is_correct else "red"))
        return is_correct

verify_pred_from_output(list_outputs[2], 135, True)

Return Sequence:
135.0

    sold_on_friday = 30
    sold_on_saturday = sold_on_friday * 2
    sold_on_sunday = sold_on_saturday - 15
    nr_boxes = sold_on_sunday + sold_on_saturday + sold_on_friday
    return float(nr_boxes)



True

In [58]:
config = hf.codegen_gen_args()
config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv
config.k = 3
config.max_lenght_after_input = 250
config.top_p = 0.95
config.top_k = 50
config.temperature = 0.7
config.min_length = 3

gen_args = config

In [14]:
tokens = tokenizer(prompt, return_tensors="pt").input_ids
generated_tokens = model.generate(
                tokens.long().cuda(),
                use_cache=True,
                do_sample=True,
                top_k=gen_args.top_k,
                temperature=gen_args.temperature,
                top_p=gen_args.top_p,
                min_length=len(tokens[0]) + gen_args.min_length,
                max_length=len(tokens[0]) + gen_args.max_length_after_input,
                num_return_sequences=gen_args.num_return_sequences,
                pad_token_id=tokenizer.eos_token_id,
            )

list_outputs = preproc_gen_toks(generated_tokens, len(tokens[0]), tokenizer)
print(list_outputs[1])


    boxes_saturday = 2
    boxes_friday = 30
    boxes_sunday = boxes_saturday - boxes_friday
    boxes_friday_sunday = boxes_friday + boxes_sunday
    nr_sold_boxes = boxes_friday_sunday * 3
    return float(nr_sold_boxes)



Return Sequence:

    sold_on_friday = 30
    sold_on_saturday = sold_on_friday * 2
    sold_on_sunday = sold_on_saturday - 15
    nr_boxes = sold_on_sunday + sold_on_saturday + sold_on_friday
    return float(nr_boxes)



False

In [63]:
exec(f"def exercise6():{list_outputs[2]}\n")
exercise6()

135.0

In [68]:
def wrapper_exec(code):
    exec(f"def exercise7():{code}\n", globals())
    return exercise7()

In [69]:
wrapper_exec("return 'pleaze'")

'pleaze'